In [1]:
import numpy as np
import numpy.matlib
import pandas as pd
import os, sys
from scipy import signal

CURRENT_DIR = os.path.dirname(os.getcwd())
sys.path.append(os.path.dirname(CURRENT_DIR+'/func'))
from func.photonic_func import Photonic

from plotly.offline import init_notebook_mode, iplot 
import plotly.graph_objs as go
init_notebook_mode(connected=True)  # for Jupyter Lab notebook

nm = 1e-9
nsec = 1e-9
c = 2.99792458e8  # [m/s] Speed of light
hc = 1.987820871E-025  # [J * m / photon] Energy of photon with wavelength m

# Create WP with n shutters per cycle

In [2]:
photonic = Photonic(config='Cfg3')

freq = 45e6
shutter_per_cycle = 7
total_shutters = shutter_per_cycle + 8

shutters = photonic.generate_tof_shutters(op=photonic.op, freq=freq, 
                                  shutter_per_cycle=shutter_per_cycle, 
                                  total_shutters=total_shutters)                                                                           
shutters['dt'].shape, shutters['sh0'].shape, 

((100,), (100,))

In [3]:
data = [None] * total_shutters # Generate blank list with length equal number of shutters
for i in range(total_shutters):
    data[i] = go.Scatter(x=shutters['dt'], y=shutters['sh'+str(i)],
                          mode='lines+markers',  # Select 'lines', 'markers' or 'lines+markers'
                          name='sh'+str(i))

layout = dict(title='WP - separated',
              xaxis=dict(title='time delay [sec]',type='linear'),  # Select 'log' or 'linear'
              yaxis=dict(title='Signal',type='linear'),  # Select 'log' or 'linear'
              template='plotly_dark')

iplot(dict(data=data, layout=layout))

In [4]:
data = [None] * shutter_per_cycle # Generate blank list with length equal number of shutters
for i in range(shutter_per_cycle):
    data[i] = go.Scatter(x=shutters['dt'], y=shutters['sh_comb'+str(i)],
                          mode='lines+markers',  # Select 'lines', 'markers' or 'lines+markers'
                          name='sh'+str(i))

layout = dict(title='WP - combined',
              xaxis=dict(title='time delay [sec]',type='linear'),  # Select 'log' or 'linear'
              yaxis=dict(title='Signal',type='linear'),  # Select 'log' or 'linear'
              template='plotly_dark')

iplot(dict(data=data, layout=layout))

# Calc phase of the phase modulation

In [5]:
phase = photonic.get_modulation_phase(shutters, shutter_per_cycle, initial_phase = -3.0)   

diff_phase = np.diff(phase) * 3.
diff_phase[np.abs(diff_phase) > 1.5] = np.nan

In [6]:
data = 2 * [None]
data[0] = go.Scatter(x=shutters['dt'], y=phase,
                          mode='lines+markers',  # Select 'lines', 'markers' or 'lines+markers'
                          name='phase')

data[1] = go.Scatter(x=shutters['dt'][:-1], y=diff_phase,
                          mode='lines+markers',  # Select 'lines', 'markers' or 'lines+markers'
                          name='phase derivative')

layout = dict(title='WP - phase',
              xaxis=dict(title='time delay [sec]', type='linear'),  # Select 'log' or 'linear'
              yaxis=dict(title='Signal', type='linear'),  # Select 'log' or 'linear'
              template='plotly_dark')

iplot(dict(data=data, layout=layout))

# Calc distance

In [7]:
shutters['dist'] = -c * shutters['dt'] / 2 + 10
data = 2 * [None]
data[0] = go.Scatter(x=shutters['dist'] , y=phase,
                          mode='lines+markers',  # Select 'lines', 'markers' or 'lines+markers'
                          name='phase')

data[1] = go.Scatter(x=shutters['dist'][:-1], y=diff_phase,
                          mode='lines+markers',  # Select 'lines', 'markers' or 'lines+markers'
                          name='phase derivative')

layout = dict(title='WP - phase',
              xaxis=dict(title='dist [m]', type='linear'),  # Select 'log' or 'linear'
              yaxis=dict(title='Signal', type='linear'),  # Select 'log' or 'linear'
              template='plotly_dark')

iplot(dict(data=data, layout=layout))
c

299792458.0

In [8]:
data = [None] * shutter_per_cycle # Generate blank list with length equal number of shutters
for i in range(shutter_per_cycle):
    data[i] = go.Scatter(x=shutters['dist'], y=shutters['sh_comb'+str(i)],
                          mode='lines+markers',  # Select 'lines', 'markers' or 'lines+markers'
                          name='sh'+str(i))

layout = dict(title='WP - combined',
              xaxis=dict(title='distance [m]',type='linear'),  # Select 'log' or 'linear'
              yaxis=dict(title='Signal',type='linear'),  # Select 'log' or 'linear'
              template='plotly_dark')

iplot(dict(data=data, layout=layout))

# Adding photonic signal - IR

In [9]:
cw_signal = photonic.photoelectron2(dist_vec=shutters['dist'])
for i in range(shutter_per_cycle):
    shutters['ir'+str(i)] = cw_signal * shutters['sh_comb'+str(i)]

In [10]:
data = [None] * (shutter_per_cycle + 1) # Generate blank list with length equal number of shutters
sum_ir = np.zeros(shutters['dist'].shape)
for i in range(shutter_per_cycle):
    sum_ir = sum_ir + shutters['ir'+str(i)]
    data[i] = go.Scatter(x=shutters['dist'], y=shutters['ir'+str(i)],
                          mode='lines+markers',  # Select 'lines', 'markers' or 'lines+markers'
                          name='sh'+str(i))
data[shutter_per_cycle] = go.Scatter(x=shutters['dist'], y=sum_ir,
                          mode='lines+markers',  # Select 'lines', 'markers' or 'lines+markers'
                          name='sum_ir')

layout = dict(title='WP - IR',
              xaxis=dict(title='distance [m]',type='linear', range=[0.4, 7]),  # Select 'log' or 'linear'
              yaxis=dict(title='Signal [photoelectron]',type='log', range=[np.log10(1), np.log10(10000)]),  # Select 'log' or 'linear'
              template='plotly_dark')

iplot(dict(data=data, layout=layout))

# Add photon shot noise

In [11]:
samples = 20
phase_per_dist = 2 * np.pi / (c/(2. * freq))
# Add Monte Carlo photon shot noise to signal
for i in range(shutter_per_cycle):
    shutters['shotN'+str(i)] = np.random.randn(samples, shutters['ir0'].shape[0]) * \
                                    np.matlib.repmat(np.sqrt(shutters['ir'+str(i)]), samples, 1) + \
                                    np.matlib.repmat(shutters['ir'+str(i)], samples, 1) 

noise_phase = photonic.get_modulation_phase(shutters, shutter_per_cycle, initial_phase = -3.0, key='shotN') 
phase_error = noise_phase - np.matlib.repmat(phase, samples, 1)
depth_error = phase_error / phase_per_dist   
depth_error.shape

(20, 100)

In [18]:
data = (samples + 1) * [None]
for i in range(samples):
    data[i] = go.Scatter(x=shutters['dist'], y=depth_error[i,:],
                              mode='markers',  # Select 'lines', 'markers' or 'lines+markers'
                              name='depth error [m]')

data[samples] = go.Scatter(x=shutters['dist'], y=np.std(depth_error, axis=0),
                              mode='lines',  # Select 'lines', 'markers' or 'lines+markers'
                              name='depth stdev [m]')

layout = dict(title='WP - Depth Error',
              xaxis=dict(title='depth [m]', type='linear', range=[0.3,7.]),  # Select 'log' or 'linear'
              yaxis=dict(title='depth error [m] ', type='linear', range=[-0.07,0.07]),  # Select 'log' or 'linear'
              template='plotly_dark')

iplot(dict(data=data, layout=layout))

# Power consumption 

In [20]:
power = photonic.power_consumption()
P_opt = power['optical'] * 1000 # [mW]
P_light = power['light'] * 1000 # [mW]
P_sensor = power['sensor'] * 1000 # [mW]
P_heat = power['heat_light'] * 1000 + P_sensor
P_total = P_light + P_sensor  
print(f'Light Power Consumption is {P_light:2.1f} mW\nSensor Power Consumption is {P_sensor:2.1f} mW')
print(f'Total Power Consumption is {P_total:2.1f} mW')
print(f'Total Heat Dissipation is {P_heat:2.1f} mW')

Light Power Consumption is 230.4 mW
Sensor Power Consumption is 165.9 mW
Total Power Consumption is 396.3 mW
Total Heat Dissipation is 304.1 mW
